# Examining the Data

In [ ]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 6
pd.options.display.max_columns = 8
pd.options.display.width = 100

In [ ]:
df = pd.read_hdf('data/beer.hdf','df')
df

In [ ]:
df.info()

# Text Data & .str accessor

http://pandas.pydata.org/pandas-docs/stable/text.html

In [ ]:
df.beer_style

In [ ]:
df.beer_style.str.len()

In [ ]:
df.beer_style.str.contains('[A|a]merican')

# Datetime Data & .dt accessor

http://pandas.pydata.org/pandas-docs/stable/timeseries.html#time-date-components

In [ ]:
df.time

In [ ]:
df.time.dt.date

In [ ]:
df.time.dt.hour

# Categoricals & .cat accessor

http://pandas.pydata.org/pandas-docs/stable/categorical.html

In [ ]:
df.select_dtypes(include=['object']).describe()

In [ ]:
df[['beer_style']].info()

In [ ]:
df['beer_style'] = df['beer_style'].astype('category')

In [ ]:
df[['beer_style']].info()

In [ ]:
df.beer_style.cat.codes

In [ ]:
df.beer_style.cat.categories

In [ ]:
df.beer_style.cat.ordered

# Indexing

http://pandas.pydata.org/pandas-docs/stable/indexing.html

# Boolean indexing

Like a where clause in SQL. The indexer (or boolean mask) should be 1-dimensional and the same length as the thing being indexed.

In [ ]:
df.abv < 5

In [ ]:
df[df.abv < 5]

Notice that we just used `[]` there. We can pass the boolean indexer in to `.loc` as well.

In [ ]:
df.loc[df.abv < 5, ['beer_style', 'review_overall']]

Again, you can get complicated

In [ ]:
df[((df.abv < 5) & (df.time > pd.Timestamp('2009-06'))) | (
        df.review_overall >= 4.5)]

# isin

Select just the rows where the `beer_style` contains IPA. 

In [ ]:
df[df.beer_style.cat.contains('IPA')]

In [ ]:
df[(df.beer_style.astype(object)).str.contains('IPA')]

### Is there another / better way?

In [ ]:
cats = (df
          .beer_style
          .cat
          .categories[df.beer_style.cat.categories.str.contains('IPA')]
        )
cats

In [ ]:
df.beer_style.isin(cats)

In [ ]:
df[df.beer_style.isin(cats)]

This is quite powerful. Any method that returns a boolean array is potentially an indexer.

In [ ]:
beer_ids = df.beer_id.value_counts()
beer_ids

In [ ]:
df[df.beer_id.isin(beer_ids[0:3].index)]

# Positional Indexing

In [ ]:
df.head(5)

In [ ]:
df.iloc[[2,5,10]]

In [ ]:
df.iloc[[2,5,10],0:3]

# Location Based Indexing

In [ ]:
df.loc[[2,5,10],['beer_id','time']]

In [ ]:
df.loc[df.beer_id.isin(beer_ids[0:3].index),['beer_id','time']]

# Questions
- why we use ``.loc``
- why do we care about uniqueness
- why do we use ``.ix``

In [ ]:
df.index.is_unique

In [ ]:
df.set_index('beer_id').index.is_unique

# Hierarchical Indexing

One of the most powerful and most complicated features of pandas.
Let's you represent high-dimensional datasets in a table.

In [ ]:
reviews = df.set_index(['profile_name', 'beer_id', 'time']).sort_index()
reviews.head()

# Why is this useful

In [ ]:
reviews.index.is_unique

In [ ]:
top_reviewers = (reviews
                        .index
                        .get_level_values('profile_name')
                        .value_counts()
                        .head(5)
                )
top_reviewers

In [ ]:
pd.options.display.max_rows=4
reviews.loc[top_reviewers.index, :, :]

In [ ]:
reviews.loc[[('BeerFMAndy',100,pd.Timestamp('2009-10-22 03:39:21'))]]

### Specifying multi-axis indexers

In [ ]:
idx = pd.IndexSlice

In [ ]:
reviews.loc[idx[top_reviewers.index, [92,54919], :], 
               ['beer_name','beer_style']]

In [ ]:
(reviews
        .query('profile_name in @top_reviewers.index and \
                beer_id in [92,54919]')
        [['beer_name','beer_style']]
 )

In [ ]:
reviews.loc[idx[:, [92, 54919], :], ['beer_name' ,'beer_style']]